# TrainLB

In [1]:
using AutomotiveDrivingModels
using DynamicBayesianNetworkBehaviors

## Parameters

In [2]:
include(Pkg.dir("AutomotiveDrivingModels", "scripts", "extract_params.jl"))
dset_filepath_modifier = "_following" # _freeflow, _following, _lanechange
dset = JLD.load(joinpath(EVALUATION_DIR, "dataset2" * dset_filepath_modifier * ".jld"),
                "model_training_data")::ModelTrainingData2
runlogs_original = load_runlogs(dset)
streetnets = load_streetnets(runlogs_original)

fold = 1
cv_split_outer = get_fold_assignment_across_drives(dset, N_FOLDS)
cv_split_inner = drop_fold!(deepcopy(cv_split_outer), fold)
@assert(cv_split_inner.nfolds > 0)

ntraces = length(cv_split_outer.seg_assignment);

## Optimize Hyperparams

In [3]:
train_def = BehaviorTrainDefinition(
                                    LB_TrainParams(
                                        indicators=INDICATOR_SET2,
                                        max_parents=10,
                                        verbosity=0,
                                    ),
                                    BehaviorParameter[
                                    ])

preallocated_data = preallocate_learning_data(dset, train_def.trainparams)
print(train_def); sleep(0.1)

BehaviorTrainDefinition
LB Train Params
	targets: (:f_des_angle,:f_accel)
	indicators: [:posFyaw,:posFt,:speed,:velBx,:velBy,:velFs,:velFt,:turnrate,:acc,:accFs,:accFt,:accBx,:accBy,:d_ml,:d_mr,:ttcr_ml,:ttcr_mr,:est_ttcr,:a_req_stayinlane,:n_lane_left,:n_lane_right,:has_lane_right,:has_lane_left,:curvature,:dist_merge,:dist_split,:scene_speed_diff,:has_front,:d_front,:d_y_front,:dv_x_front,:dv_y_front,:yaw_front,:turnrate_front,:acc_req_front,:inv_ttc_front,:inv_timegap_front,:gaining_on_front,:has_rear,:d_rear,:d_y_rear,:dv_x_rear,:dv_y_rear,:yaw_rear,:turnrate_rear,:acc_req_rear,:inv_ttc_rear,:inv_timegap_rear,:gaining_on_rear,:has_left,:d_left,:d_y_left,:dv_x_left,:dv_y_left,:yaw_left,:turnrate_left,:acc_req_left,:inv_ttc_left,:inv_timegap_left,:left_is_gaining,:has_right,:d_right,:d_y_right,:dv_x_right,:dv_y_right,:yaw_right,:turnrate_right,:acc_req_right,:inv_ttc_right,:inv_timegap_right,:right_is_gaining,:past_1_velFt,:past_2_velFt,:past_3_velFt,:past_4_velFt,:past_1_posFt,:past

In [4]:
preallocated_data.features_disc

12-element Array{AutomotiveDrivingModels.Features.AbstractFeature,1}:
 AutomotiveDrivingModels.Features.Feature_N_Lane_Left()     
 AutomotiveDrivingModels.Features.Feature_N_Lane_Right()    
 AutomotiveDrivingModels.Features.Feature_Has_Lane_Right()  
 AutomotiveDrivingModels.Features.Feature_Has_Lane_Left()   
 AutomotiveDrivingModels.Features.Feature_Has_Front()       
 AutomotiveDrivingModels.Features.Feature_Gaining_On_Front()
 AutomotiveDrivingModels.Features.Feature_Has_Rear()        
 AutomotiveDrivingModels.Features.Feature_Rear_Is_Gaining() 
 AutomotiveDrivingModels.Features.Feature_Has_Left()        
 AutomotiveDrivingModels.Features.Feature_Left_Is_Gaining() 
 AutomotiveDrivingModels.Features.Feature_Has_Right()       
 AutomotiveDrivingModels.Features.Feature_Right_Is_Gaining()

In [5]:
preallocated_data.features_cont

19-element Array{AutomotiveDrivingModels.Features.AbstractFeature,1}:
 AutomotiveDrivingModels.Features.Feature_PosFyaw()               
 AutomotiveDrivingModels.Features.Feature_PosFt()                 
 AutomotiveDrivingModels.Features.Feature_Speed()                 
 AutomotiveDrivingModels.Features.Feature_VelBx()                 
 AutomotiveDrivingModels.Features.Feature_VelBy()                 
 AutomotiveDrivingModels.Features.Feature_VelFs()                 
 AutomotiveDrivingModels.Features.Feature_VelFt()                 
 AutomotiveDrivingModels.Features.Feature_TurnRate()              
 AutomotiveDrivingModels.Features.Feature_Acc()                   
 AutomotiveDrivingModels.Features.Feature_AccFs()                 
 AutomotiveDrivingModels.Features.Feature_AccFt()                 
 AutomotiveDrivingModels.Features.Feature_AccBx()                 
 AutomotiveDrivingModels.Features.Feature_AccBy()                 
 AutomotiveDrivingModels.Features.Feature_MarkerDist_Left()

In [6]:
preallocated_data.features_nann

76-element Array{AutomotiveDrivingModels.Features.AbstractFeature,1}:
 AutomotiveDrivingModels.Features.Feature_TimeToCrossing_Left()             
 AutomotiveDrivingModels.Features.Feature_TimeToCrossing_Right()            
 AutomotiveDrivingModels.Features.Feature_EstimatedTimeToLaneCrossing()     
 AutomotiveDrivingModels.Features.Feature_A_REQ_StayInLane()                
 AutomotiveDrivingModels.Features.Feature_Dist_Front()                      
 AutomotiveDrivingModels.Features.Feature_D_Y_Front()                       
 AutomotiveDrivingModels.Features.Feature_Delta_V_Front()                   
 AutomotiveDrivingModels.Features.Feature_Delta_V_Y_Front()                 
 AutomotiveDrivingModels.Features.Feature_Yaw_Front()                       
 AutomotiveDrivingModels.Features.Feature_Turnrate_Front()                  
 AutomotiveDrivingModels.Features.Feature_Acc_Req_Front()                   
 AutomotiveDrivingModels.Features.Feature_Inv_TTC_Front()                   
 Autom

In [7]:
# AutomotiveDrivingModels.optimize_hyperparams_cyclic_coordinate_ascent!(
#                     train_def, dset, preallocated_data, cv_split_inner)
# print(train_def); sleep(0.1)

## Train

In [11]:
LB = train(dset, preallocated_data, train_def.trainparams, FoldSet(cv_split_outer, fold, false, :frame))
sleep(0.5)
println("\n\n")
print(LB)
sleep(0.5)




Linear Gaussian Bayesian Network Behavior
	targets: (:f_des_angle,:f_accel)
	features: 
		discrete:   [:gaining_on_front,:has_rear,:has_left,:right_is_gaining]
		continuous: [:acc,:accFt,:accBx,:est_ttcr,:past_1_acc,:past_2_acc,:past_3_acc,:past_1_f_des_angle,:past_2_f_des_angle,:mean_8_acc,:mean_8_f_des_angle,:max_4_acc,:f_des_angle,:f_accel]
	number of suffstats for lat: 1280
	number of suffstats for lon: 208
	parents lat: [:gaining_on_front,:has_rear,:has_left,:right_is_gaining]
	             [:accFt]
	             [:est_ttcr,:past_1_f_des_angle,:past_2_f_des_angle,:mean_8_f_des_angle]
	parents lon: Symbol[]
	             [:acc,:accBx]
	             [:past_1_acc,:past_2_acc,:past_3_acc,:mean_8_acc,:max_4_acc]
	sample lat first: true


## Likelihood

In [12]:
arr_logl_train = Float64[]
arr_logl_test = Float64[]
for frameind in 1 : nrow(dset.dataframe)
    logl = calc_action_loglikelihood(LB, dset.dataframe, frameind)
    
    if is_in_fold(fold, cv_split_outer.frame_assignment[frameind], true)
        push!(arr_logl_test, logl)
    else
        push!(arr_logl_train, logl)
    end
end

@printf("%20s  %12s  %12s  %12s\n", "min", "mean", "median", "max")
@printf("%20.4f  %12.4f  %12.4f  %12.4f\n", minimum(arr_logl_train), mean(arr_logl_train), median(arr_logl_train), maximum(arr_logl_train))
@printf("%20.4f  %12.4f  %12.4f  %12.4f\n", minimum(arr_logl_test), mean(arr_logl_test), median(arr_logl_test), maximum(arr_logl_test))

                 min          mean        median           max
 -1853193673396.4333  -113828818.4528       15.5193       16.8815
        -189877.3036      -36.5944       15.2974       16.8832
